In [1]:
import numpy as np
from tensorflow import keras
import os
import cv2
from os import listdir
from os.path import isfile
from os.path import join
import time

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Data Processing

In [2]:
(train_data, train_label), (test_data, test_label) = keras.datasets.mnist.load_data()
train_data = train_data.reshape(-1,28,28,1)
test_data = test_data.reshape(-1,28,28,1)

train_label = to_categorical(train_label, 10)
test_label = to_categorical(test_label, 10)


train_datagen=ImageDataGenerator(rescale=1./255, 
                                 #rotation_range = 15, 
                                 width_shift_range = 0.2, 
                                 height_shift_range = 0.2,
                                 shear_range = 0.2,
                                 zoom_range = 0.2)



test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen.fit(train_data)
test_datagen.fit(test_data)

LR_function = ReduceLROnPlateau(monitor = 'val_loss',
                                patience = 2,
                                verbose = 1,
                                factor = 0.5,
                                min_lr = 0.00001)

EarlyStop = EarlyStopping(monitor='val_loss',
                          patience = 6,
                          verbose = 1, 
                          mode = 'min')

print(np.array(train_data).shape)
print(np.array(train_label).shape)

(60000, 28, 28, 1)
(60000, 10)


# CNN-Mnist

In [9]:
# model 
model = Sequential()

#conv1
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal", activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())

#conv2
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#pooling1
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#conv3
model.add(Conv2D(256,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#conv4
model.add(Conv2D(256,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#pooling2
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#conv5
model.add(Conv2D(256,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#conv6
model.add(Conv2D(512,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#conv7
model.add(Conv2D(512,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#pooling3
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#conv8
model.add(Conv2D(512,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#conv9
model.add(Conv2D(512,(3,3), padding="same", kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())

#pooling4
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(512, kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(256, kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(10, activation='softmax'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 28, 28, 128)       1280      
_________________________________________________________________
batch_normalization_22 (Batc (None, 28, 28, 128)       512       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 128)       147584    
_________________________________________________________________
batch_normalization_23 (Batc (None, 28, 28, 128)       512       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 14, 14, 256)      

# CNN Evaluation

In [10]:
epoch = 50
batch_size = 32

model.compile(
    loss = "categorical_crossentropy", 
    optimizer = "Adam",
    metrics = ["accuracy"]
)

history = model.fit(train_datagen.flow(train_data,train_label, batch_size = batch_size), 
                    steps_per_epoch = train_data.shape[0] / batch_size , 
                    validation_data = test_datagen.flow(test_data, test_label, batch_size = batch_size),
                    validation_steps = test_data.shape[0] / batch_size,
                    epochs = epoch,
                    callbacks = [LR_function, EarlyStop])

Epoch 1/50
1875/1875 [==============================] - 32s 16ms/step - loss: 0.3993 - accuracy: 0.8799 - val_loss: 0.0773 - val_accuracy: 0.9769
Epoch 2/50
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1109 - accuracy: 0.9689 - val_loss: 0.0352 - val_accuracy: 0.9901
Epoch 3/50
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0948 - accuracy: 0.9738 - val_loss: 0.0373 - val_accuracy: 0.9890
Epoch 4/50
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0816 - accuracy: 0.9783 - val_loss: 0.0349 - val_accuracy: 0.9896
Epoch 5/50
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0695 - accuracy: 0.9817 - val_loss: 0.0319 - val_accuracy: 0.9907
Epoch 6/50
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0638 - accuracy: 0.9842 - val_loss: 0.0270 - val_accuracy: 0.9924
Epoch 7/50
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0542 - accuracy: 0.9857 - val_loss: 0.0241 -

# BP-Mnist

In [3]:
# model 
model = Sequential()

model.add(Flatten(input_shape=[28, 28, 1]))

model.add(Dense(512, kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(256, kernel_initializer="he_normal", activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(10, activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0

# BP Evaluation 

In [4]:
epoch = 50
batch_size = 32

model.compile(
    loss = "categorical_crossentropy", 
    optimizer = "Adam",
    metrics = ["accuracy"]
)

history = model.fit(train_datagen.flow(train_data,train_label, batch_size = batch_size), 
                    steps_per_epoch = train_data.shape[0] / batch_size , 
                    validation_data = test_datagen.flow(test_data, test_label, batch_size = batch_size),
                    validation_steps = test_data.shape[0] / batch_size,
                    epochs = epoch,
                    callbacks = [LR_function, EarlyStop])

Epoch 1/50
1875/1875 [==============================] - 12s 6ms/step - loss: 0.9897 - accuracy: 0.6751 - val_loss: 0.2677 - val_accuracy: 0.9148
Epoch 2/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.5972 - accuracy: 0.8065 - val_loss: 0.1993 - val_accuracy: 0.9375
Epoch 3/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.5078 - accuracy: 0.8381 - val_loss: 0.1680 - val_accuracy: 0.9485
Epoch 4/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.4524 - accuracy: 0.8554 - val_loss: 0.1323 - val_accuracy: 0.9587ETA: 2s
Epoch 5/50
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4241 - accuracy: 0.8657 - val_loss: 0.1321 - val_accuracy: 0.9593
Epoch 6/50
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3925 - accuracy: 0.8754 - val_loss: 0.1333 - val_accuracy: 0.9584
Epoch 7/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3754 - accuracy: 0.8792 - val_loss: 0.1259 -